In [ ]:
'Ich probier jetzt jeden Schritt nochmal von vorne und schau mir den Schritt nochmal genauer an'

from wettbewerb import load_references, get_6montages
from new_preprocess import preprocess_signal_with_montages


ids, channels_list, data_list, fs_list, ref_list, label_list = load_references(folder="../shared_data/training", idx=0)
for i in range(100):
    #montage_names, montage_data, missing = get_6montages(channels_list[i], data_list[i])
    #print(f"{ids[i]}:{montage_names}:{montage_data.shape}\n {fs_list[i]}")
    #if missing:
        #print("Warning: Montage missing, data may be incomplete.")
    #print (f"{ids[i]}",label_list[i])
    processed_signal, montage_missing = preprocess_signal_with_montages(channels_list[i], data_list[i], 256, fs_list[i])
    print(f"{ids[i]}:",processed_signal.shape)

In [ ]:
from wettbewerb import EEGDataset

dataset = EEGDataset("../shared_data/training")
labels = dataset.get_labels()

# Count seizure / non-seizure
seizure_count = sum(1 for l in labels if l[0])
non_seizure_count = len(labels) - seizure_count

print(f"Total: {len(labels)}")
print(f"Seizures: {seizure_count}")
print(f"Non-Seizures: {non_seizure_count}")

In [1]:
from wettbewerb import EEGDataset
import os
import torch 
from new_preprocess import preprocess_signal_with_montages
dataset = EEGDataset("../shared_data/training")
save_folder = "preprocessed_data"
os.makedirs(save_folder, exist_ok=True)

processed_count = 0
skipped_count = 0
total = len(dataset)

for i in range(total):
    ids, channels, data, fs, ref, label = dataset[i]

    montage_names, processed_signal, montage_missing, resampled_fs = preprocess_signal_with_montages(
        channels, data, target_fs=256, original_fs=fs, ids=ids
    )

    if montage_missing:
        skipped_count += 1
        print(f"[{i+1}/{total}] Skipping {ids} (montage missing)")
        continue

    save_path = os.path.join(save_folder, f"{ids}.pt")
    torch.save((processed_signal, label, ids, montage_names, resampled_fs), save_path)
    processed_count += 1
    print(f"[{i+1}/{total}] Processed: {processed_count} | Skipped: {skipped_count}", end='\r')

print("Done!")

Done!/6213] Processed: 6213 | Skipped: 0


In [1]:
from wettbewerb import EEGDataset
from new_preprocess import preprocess_signal_with_montages
from new_features import window_eeg_data, feature_extraction_window  # your modules
import os, torch
import numpy as np

window_size = 4  # seconds
step_size = 2    # seconds

dataset = EEGDataset("../shared_data/training")
save_folder = f"montage_datasets/win{window_size}_step{step_size}"
os.makedirs(save_folder, exist_ok=True)

for i in range(len(dataset)):
    eeg_id, channels, raw_data, fs, _, label = dataset[i]
    seizure_label, seizure_onset, seizure_offset = label

    # 1. Preprocess
    montage_names, processed_signal, montage_missing, new_fs = preprocess_signal_with_montages(
        channels, raw_data, target_fs=256, original_fs=fs, ids=eeg_id
    )

    if montage_missing:
        print(f"Skipping {eeg_id} (montage missing)")
        continue

    # 2. Windowing + labeling
    windows, labels, timestamps = window_eeg_data(
        processed_signal, resampled_fs=new_fs,
        seizure_onset=seizure_onset,
        seizure_offset=seizure_offset,
        window_size=window_size,
        step_size=step_size
    )

    # 3. Feature extraction per window
    for idx, (window, lbl, ts) in enumerate(zip(windows, labels, timestamps)):
        features = feature_extraction_window(window, new_fs)
        save_path = os.path.join(save_folder, f"{eeg_id}_win{idx}_lbl{lbl}.pt")
        torch.save((features, lbl, eeg_id, ts), save_path)

    print(f"[{i+1}/{len(dataset)}] Processed {eeg_id} with {len(windows)} windows.", end='\r')

KeyboardInterrupt: 